# Marine Heatwaves and Cold Spells

Authors: Kate Collingridge and Lenka Fronkova

Institution: CEFAS

This script flags near real time sea surface temperature (SST) data to determine whether it falls outside the 10th or 90th percentile for each day of year to determine unusually warm or cold days. It then calculates the number of consecutive flagged days up to the most recent day in a 5 or 10 day window to evaluate whether there is a Marine Heatwave or Cold Spell (5 or more consecutive days flagged), or to indicate the possibility of one developing (3 or 4 days consecutive days flagged)

The resulting products are available to view as datacubes on [Cefas Data Cube Viewer](https://eutro-cube.cefas.co.uk/)

## Import packages and functions

In [351]:
import MHWCS_functions as mhwcs
import xarray as xr
import pandas as pd
import numpy as np

## Set working directory

In [3]:
dir = "C:/Users/KC05/OneDrive - CEFAS/scripts/DCS4COP/"

## Day of year climatology

This climatology is created using _scriptname_, which uses 10 years of SST data from CMEMS product [ST_GLO_SST_L4_REP_OBSERVATIONS_010_011](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=SST_GLO_SST_L4_REP_OBSERVATIONS_010_011) to calculate a climatology of the 10th and 90th percentile for each day of year.

In [4]:
ds_qt10 = xr.open_mfdataset(dir + 'MarineHeatawaves/cmems_ostia_clim/qt10/*.nc', concat_dim = 'time')
ds_qt90 = xr.open_mfdataset(dir + 'MarineHeatawaves/cmems_ostia_clim/qt90/*.nc', concat_dim = 'time')

## NRT SST data

NRT SST data are from CMEMS product [SST_GLO_SST_L4_NRT_OBSERVATIONS_010_001](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=SST_GLO_SST_L4_NRT_OBSERVATIONS_010_001), downloaded via [FTP](nrt.cmems-du.eu//Core/SST_GLO_SST_L4_NRT_OBSERVATIONS_010_001/METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2)

In [5]:
sst_NRT = xr.open_mfdataset(dir +'cmems_ostia/*/*.nc', concat_dim='time')

Subsetting NRT data to area of interest. This needs to be the same as in the climatology.

In [7]:
sst_NRT = sst_NRT.where((sst_NRT.lon >-10.4) & (sst_NRT.lon < 10.4) & (sst_NRT.lat >44.8) & (sst_NRT.lat < 65.6), drop=True)

## Flagging

Flagging NRT data using climatology.

In [109]:
flags = mhwcs.Flagging3D(sst_xar=sst_NRT, quantile_xar_cold = ds_qt10, quantile_xar_warm = ds_qt90)

Rechunking variables and adding attributes

In [205]:
flags['warm_flags'] = flags['warm_flags'].chunk(chunks = {'time': (1), 'lat': 416, 'lon': 416})
flags['cold_flags'] = flags['cold_flags'].chunk(chunks = {'time': (1), 'lat': 416, 'lon': 416})
flags['warm_flags'].attrs['long_name'] = 'Daily MHW Flag'
flags['warm_flags'].attrs['description'] = 'Daily warm spell flags where the temperature is greater than the 90th percentile climatology'
flags['cold_flags'].attrs['long_name'] = 'Daily MCS Flag'
flags['cold_flags'].attrs['description'] = 'Daily cold spell flags where the temperature is less than the 10th percentile climatology'

Exporting flags as daily files to be made into a datacube using xcube gen. It reads time (and start and stop time) from the global attributes, so these are added here.

In [15]:
time = flags['time'].values
for t in time:
    print(str(t)[0:4]+str(t)[5:7]+str(t)[8:10])
    flags_day = flags.sel(time = t)
    flags_day = flags_day.expand_dims('time')
    flags_day.attrs['time_start'] = str(pd.Timestamp(t))+' UTC'
    flags_day.attrs['time_stop'] = str(pd.Timestamp(t))+' UTC'
    flags_day.attrs['time_coverage_start'] = str(t)[0:4]+str(t)[5:7]+str(t)[8:10]+'T000000Z'
    flags_day.attrs['start_time'] = str(t)[0:4]+str(t)[5:7]+str(t)[8:10]+'T000000Z'
    flags_day.attrs['time_coverage_end'] = str(t+np.timedelta64(1,'D'))[0:4]+str(t+np.timedelta64(1,'D'))[5:7]+str(t+np.timedelta64(1,'D'))[8:10]+'T000000Z'
    flags_day.attrs['stop_time'] = str(t+np.timedelta64(1,'D'))[0:4]+str(t+np.timedelta64(1,'D'))[5:7]+str(t+np.timedelta64(1,'D'))[8:10]+'T000000Z'
    flags_day.to_netcdf(dir +'MarineHeatawaves/cmems_ostia_clim/flags/flags_daily_{0}.nc'.format(str(t)[0:4]+str(t)[5:7]+str(t)[8:10]))


20210101
20210102
20210103
20210104
20210105
20210106
20210107
20210108
20210109
20210110
20210111
20210112
20210113
20210114
20210115
20210116
20210117
20210118
20210119
20210120
20210121
20210122
20210123
20210124
20210125
20210126
20210127
20210128
20210129
20210130
20210131
20210201
20210202
20210203
20210204
20210205
20210206
20210207
20210208
20210209
20210210
20210211
20210212
20210213
20210214
20210215
20210216
20210217
20210218
20210219
20210220
20210221
20210222
20210223
20210224
20210225
20210226
20210227
20210228
20210301
20210302
20210303
20210304
20210305
20210306
20210307
20210308
20210309
20210310
20210311
20210312
20210313
20210314
20210315
20210316
20210317
20210318
20210319
20210320
20210321
20210322
20210323


## Warm/cold spell duration

Calculating duration of warm and cold spells within 5 and 10 day windows

In [182]:
warmspell5d = mhwcs.warmspelldur(flags, window = 5, flags = 'warm_flags')

In [185]:
warmspell5d

<xarray.Dataset>
Dimensions:       (lat: 416, lon: 416, time: 82)
Coordinates:
  * time          (time) datetime64[ns] 2021-01-01T12:00:00 ... 2021-03-23T12...
  * lat           (lat) float32 44.83 44.88 44.92 44.97 ... 65.47 65.53 65.57
  * lon           (lon) float32 -10.38 -10.32 -10.27 ... 10.27 10.32 10.38
Data variables:
    warmspelldur  (time, lat, lon) float64 nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:                CF-1.4, ACDD-1.3
    title:                      Global SST & Sea Ice Analysis, L4 OSTIA, 0.05...
    summary:                    A merged, multi-sensor L4 Foundation SST product
    references:                 Good S, Fiedler E, Mao C, Martin MJ, Maycock ...
    institution:                UKMO
    history:                    Created from sst.nc; obs_anal.nc; seaice.nc
    comment:                    WARNING Some applications are unable to prope...
    license:                    These data are available free of charge under...
    id:                         OSTIA-UKMO-L4-GLOB-v2.0
    naming_authority:           org.ghrsst
    product_version:            3.4
    uuid:                       536d4865-f5a8-45b2-806f-1f1db491069a
    gds_version_id:             2.4
    netcdf_version_id:          4.1
    date_created:               20210102T063658Z
    start_time:                 20210101T000000Z
    time_coverage_start:        20210101T000000Z
    stop_time:                  20210102T000000Z
    time_coverage_end:          20210102T000000Z
    file_quality_level:         3
    source:                     AVHRR18_G-NAVO-L2P-V1.0, AVHRR19_G-NAVO-L2P-V...
    platform:                   NOAA-18, NOAA-19, MetOpB, NPP, GCOM-W, GOES13...
    sensor:                     AVHRR, VIIRS, AMSR2, GOES_IMAGER, SEVIRI, SSM...
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    westernmost_longitude:      -180.0
    easternmost_longitude:      180.00002
    southernmost_latitude:      -90.0
    northernmost_latitude:      90.0
    spatial_resolution:         0.05 degree
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.05
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.05
    acknowledgment:             Please acknowledge the use of these data with...
    creator_name:               Met Office as part of CMEMS
    creator_email:              servicedesk.cmems@mercator-ocean.eu
    creator_url:                http://marine.copernicus.eu
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [186]:
coldspell5d = mhwcs.coldspelldur(flags, window = 5, flags = 'cold_flags')

In [187]:
coldspell5d

<xarray.Dataset>
Dimensions:       (lat: 416, lon: 416, time: 82)
Coordinates:
  * time          (time) datetime64[ns] 2021-01-01T12:00:00 ... 2021-03-23T12...
  * lat           (lat) float32 44.83 44.88 44.92 44.97 ... 65.47 65.53 65.57
  * lon           (lon) float32 -10.38 -10.32 -10.27 ... 10.27 10.32 10.38
Data variables:
    coldspelldur  (time, lat, lon) float64 nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:                CF-1.4, ACDD-1.3
    title:                      Global SST & Sea Ice Analysis, L4 OSTIA, 0.05...
    summary:                    A merged, multi-sensor L4 Foundation SST product
    references:                 Good S, Fiedler E, Mao C, Martin MJ, Maycock ...
    institution:                UKMO
    history:                    Created from sst.nc; obs_anal.nc; seaice.nc
    comment:                    WARNING Some applications are unable to prope...
    license:                    These data are available free of charge under...
    id:                         OSTIA-UKMO-L4-GLOB-v2.0
    naming_authority:           org.ghrsst
    product_version:            3.4
    uuid:                       536d4865-f5a8-45b2-806f-1f1db491069a
    gds_version_id:             2.4
    netcdf_version_id:          4.1
    date_created:               20210102T063658Z
    start_time:                 20210101T000000Z
    time_coverage_start:        20210101T000000Z
    stop_time:                  20210102T000000Z
    time_coverage_end:          20210102T000000Z
    file_quality_level:         3
    source:                     AVHRR18_G-NAVO-L2P-V1.0, AVHRR19_G-NAVO-L2P-V...
    platform:                   NOAA-18, NOAA-19, MetOpB, NPP, GCOM-W, GOES13...
    sensor:                     AVHRR, VIIRS, AMSR2, GOES_IMAGER, SEVIRI, SSM...
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    westernmost_longitude:      -180.0
    easternmost_longitude:      180.00002
    southernmost_latitude:      -90.0
    northernmost_latitude:      90.0
    spatial_resolution:         0.05 degree
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.05
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.05
    acknowledgment:             Please acknowledge the use of these data with...
    creator_name:               Met Office as part of CMEMS
    creator_email:              servicedesk.cmems@mercator-ocean.eu
    creator_url:                http://marine.copernicus.eu
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [174]:
warmspell10d = mhwcs.warmspelldur(flags, window = 10, flags = 'warm_flags')

In [193]:
warmspell10d

<xarray.Dataset>
Dimensions:       (lat: 416, lon: 416, time: 82)
Coordinates:
  * time          (time) datetime64[ns] 2021-01-01T12:00:00 ... 2021-03-23T12...
  * lat           (lat) float32 44.83 44.88 44.92 44.97 ... 65.47 65.53 65.57
  * lon           (lon) float32 -10.38 -10.32 -10.27 ... 10.27 10.32 10.38
Data variables:
    warmspelldur  (time, lat, lon) float64 nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:                CF-1.4, ACDD-1.3
    title:                      Global SST & Sea Ice Analysis, L4 OSTIA, 0.05...
    summary:                    A merged, multi-sensor L4 Foundation SST product
    references:                 Good S, Fiedler E, Mao C, Martin MJ, Maycock ...
    institution:                UKMO
    history:                    Created from sst.nc; obs_anal.nc; seaice.nc
    comment:                    WARNING Some applications are unable to prope...
    license:                    These data are available free of charge under...
    id:                         OSTIA-UKMO-L4-GLOB-v2.0
    naming_authority:           org.ghrsst
    product_version:            3.4
    uuid:                       536d4865-f5a8-45b2-806f-1f1db491069a
    gds_version_id:             2.4
    netcdf_version_id:          4.1
    date_created:               20210102T063658Z
    start_time:                 20210101T000000Z
    time_coverage_start:        20210101T000000Z
    stop_time:                  20210102T000000Z
    time_coverage_end:          20210102T000000Z
    file_quality_level:         3
    source:                     AVHRR18_G-NAVO-L2P-V1.0, AVHRR19_G-NAVO-L2P-V...
    platform:                   NOAA-18, NOAA-19, MetOpB, NPP, GCOM-W, GOES13...
    sensor:                     AVHRR, VIIRS, AMSR2, GOES_IMAGER, SEVIRI, SSM...
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    westernmost_longitude:      -180.0
    easternmost_longitude:      180.00002
    southernmost_latitude:      -90.0
    northernmost_latitude:      90.0
    spatial_resolution:         0.05 degree
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.05
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.05
    acknowledgment:             Please acknowledge the use of these data with...
    creator_name:               Met Office as part of CMEMS
    creator_email:              servicedesk.cmems@mercator-ocean.eu
    creator_url:                http://marine.copernicus.eu
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [169]:
coldspell10d = mhwcs.coldspelldur(flags, window = 10, flags = 'cold_flags')

In [170]:
coldspell10d

<xarray.Dataset>
Dimensions:       (lat: 416, lon: 416, time: 82)
Coordinates:
  * time          (time) datetime64[ns] 2021-01-01T12:00:00 ... 2021-03-23T12...
  * lat           (lat) float32 44.83 44.88 44.92 44.97 ... 65.47 65.53 65.57
  * lon           (lon) float32 -10.38 -10.32 -10.27 ... 10.27 10.32 10.38
Data variables:
    coldspelldur  (time, lat, lon) float64 nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:                CF-1.4, ACDD-1.3
    title:                      Global SST & Sea Ice Analysis, L4 OSTIA, 0.05...
    summary:                    A merged, multi-sensor L4 Foundation SST product
    references:                 Good S, Fiedler E, Mao C, Martin MJ, Maycock ...
    institution:                UKMO
    history:                    Created from sst.nc; obs_anal.nc; seaice.nc
    comment:                    WARNING Some applications are unable to prope...
    license:                    These data are available free of charge under...
    id:                         OSTIA-UKMO-L4-GLOB-v2.0
    naming_authority:           org.ghrsst
    product_version:            3.4
    uuid:                       536d4865-f5a8-45b2-806f-1f1db491069a
    gds_version_id:             2.4
    netcdf_version_id:          4.1
    date_created:               20210102T063658Z
    start_time:                 20210101T000000Z
    time_coverage_start:        20210101T000000Z
    stop_time:                  20210102T000000Z
    time_coverage_end:          20210102T000000Z
    file_quality_level:         3
    source:                     AVHRR18_G-NAVO-L2P-V1.0, AVHRR19_G-NAVO-L2P-V...
    platform:                   NOAA-18, NOAA-19, MetOpB, NPP, GCOM-W, GOES13...
    sensor:                     AVHRR, VIIRS, AMSR2, GOES_IMAGER, SEVIRI, SSM...
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    westernmost_longitude:      -180.0
    easternmost_longitude:      180.00002
    southernmost_latitude:      -90.0
    northernmost_latitude:      90.0
    spatial_resolution:         0.05 degree
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.05
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.05
    acknowledgment:             Please acknowledge the use of these data with...
    creator_name:               Met Office as part of CMEMS
    creator_email:              servicedesk.cmems@mercator-ocean.eu
    creator_url:                http://marine.copernicus.eu
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

Combining everything into one dataset

In [342]:
spelldur = xr.Dataset(
        data_vars = dict(
            warm_flags = (["time", "lat", "lon"], flags['warm_flags'].values, flags['warm_flags'].attrs),
            cold_flags = (["time", "lat", "lon"], flags['cold_flags'].values, flags['cold_flags'].attrs),
            warm_spell_dur_5days = (["time", "lat", "lon"], warmspell5d['warmspelldur'].values),
            cold_spell_dur_5days = (["time", "lat", "lon"], coldspell5d['coldspelldur'].values),
            warm_spell_dur_10days = (["time", "lat", "lon"], warmspell10d['warmspelldur'].values),
            cold_spell_dur_10days = (["time", "lat", "lon"], coldspell10d['coldspelldur'].values)
        ),
        coords = dict(
            lon = flags['lon'],
            lat = flags['lat'],
            time = flags['time']
        ),
        attrs = dict(description = 'Products for prediction of marine heatwaves and cold spells in near-real-time',
                   institution = 'CEFAS',
                   creator_name = 'Kate Collingridge and Lenka Fronkova',
                   version = '1.0',
                   comment = 'Please visit the GitHub repository for more information on the methods used to create these products.',
                   github_repo = 'https://github.com/CefasRepRes/MHWCS_warning/tree/main'
        ),
    )


Adding attributes and rechunking

In [343]:
# from my investigation units = days is not allowed for some reason so I have changed to units = day

spelldur['warm_spell_dur_5days'].attrs['long_name'] = 'MHW Warning'
spelldur['warm_spell_dur_5days'].attrs['description'] = 'Near real time mapping of MHW.  Value 5 signifies five consecutive days of anomalously warm water, which marks an onset of a heatwave event. Values 4 and 3 signify four or three consecutive days of anomalously warm water. There is a potential of a heatwave developing if this continues in the following one or two days respectively.'
spelldur['warm_spell_dur_5days'].attrs['units'] = 'day'

spelldur['cold_spell_dur_5days'].attrs['long_name'] = 'MCS Warning'
spelldur['cold_spell_dur_5days'].attrs['description'] = 'Near-real-time mapping of MCS.  Value 5 signifies five consecutive days of anomalously cold water, which marks an onset of a cold spell. Values 4 and 3 signify four or three consecutive days of anomalously cold water. There is a potential of a cold spell developing if this continues in the following one or two days respectively.'
spelldur['cold_spell_dur_5days'].attrs['units'] = 'day'

spelldur['warm_spell_dur_10days'].attrs['long_name'] = 'MHW Duration'
spelldur['warm_spell_dur_10days'].attrs['description'] = 'Marine heatwave duration in 10 days rolling window'
spelldur['warm_spell_dur_10days'].attrs['units'] = 'day'

spelldur['cold_spell_dur_10days'].attrs['long_name'] = 'MCS Duration'
spelldur['cold_spell_dur_10days'].attrs['description'] = 'Cold spell duration in 10 days rolling window'
spelldur['cold_spell_dur_10days'].attrs['units'] = 'day'


In [344]:
spelldur['warm_flags'] = spelldur['warm_flags'].chunk(chunks = {'time': (1), 'lat': 416, 'lon': 416})
spelldur['cold_flags'] = spelldur['cold_flags'].chunk(chunks = {'time': (1), 'lat': 416, 'lon': 416})
spelldur['warm_spell_dur_5days'] = spelldur['warm_spell_dur_5days'].chunk(chunks = {'time': (1), 'lat': 416, 'lon': 416})
spelldur['cold_spell_dur_5days'] = spelldur['cold_spell_dur_5days'].chunk(chunks = {'time': (1), 'lat': 416, 'lon': 416})
spelldur['warm_spell_dur_10days'] = spelldur['warm_spell_dur_10days'].chunk(chunks = {'time': (1), 'lat': 416, 'lon': 416})
spelldur['cold_spell_dur_10days'] = spelldur['cold_spell_dur_10days'].chunk(chunks = {'time': (1), 'lat': 416, 'lon': 416})

In [345]:
spelldur

<xarray.Dataset>
Dimensions:                (lat: 416, lon: 416, time: 82)
Coordinates:
  * lon                    (lon) float32 -10.38 -10.32 -10.27 ... 10.32 10.38
  * lat                    (lat) float32 44.83 44.88 44.92 ... 65.47 65.53 65.57
  * time                   (time) datetime64[ns] 2021-01-01T12:00:00 ... 2021...
Data variables:
    warm_flags             (time, lat, lon) float64 dask.array<chunksize=(1, 416, 416), meta=np.ndarray>
    cold_flags             (time, lat, lon) float64 dask.array<chunksize=(1, 416, 416), meta=np.ndarray>
    warm_spell_dur_5days   (time, lat, lon) float64 dask.array<chunksize=(1, 416, 416), meta=np.ndarray>
    cold_spell_dur_5days   (time, lat, lon) float64 dask.array<chunksize=(1, 416, 416), meta=np.ndarray>
    warm_spell_dur_10days  (time, lat, lon) float64 dask.array<chunksize=(1, 416, 416), meta=np.ndarray>
    cold_spell_dur_10days  (time, lat, lon) float64 dask.array<chunksize=(1, 416, 416), meta=np.ndarray>
Attributes:
    description:   Products for prediction of marine heatwaves and cold spell...
    institution:   CEFAS
    creator_name:  Kate Collingridge and Lenka Fronkova
    version:       1.0
    comment:       Please visit the GitHub repository for more information on...
    github_repo:   https://github.com/CefasRepRes/MHWCS_warning/tree/main

Exporting flags as daily files to be made into a datacube using xcube gen. It reads time (and start and stop time) from the global attributes, so these are added here.

In [349]:
time = spelldur['time'].values
for t in time:
    print(str(t)[0:4]+str(t)[5:7]+str(t)[8:10])
    spelldur_day = spelldur.sel(time = t)
    spelldur_day = spelldur_day.expand_dims('time')
    spelldur_day.attrs['time_start'] = str(pd.Timestamp(t))+' UTC'
    spelldur_day.attrs['time_stop'] = str(pd.Timestamp(t))+' UTC'
    spelldur_day.attrs['time_coverage_start'] = str(t)[0:4]+str(t)[5:7]+str(t)[8:10]+'T000000Z'
    spelldur_day.attrs['start_time'] = str(t)[0:4]+str(t)[5:7]+str(t)[8:10]+'T000000Z'
    spelldur_day.attrs['time_coverage_end'] = str(t+np.timedelta64(1,'D'))[0:4]+str(t+np.timedelta64(1,'D'))[5:7]+str(t+np.timedelta64(1,'D'))[8:10]+'T000000Z'
    spelldur_day.attrs['stop_time'] = str(t+np.timedelta64(1,'D'))[0:4]+str(t+np.timedelta64(1,'D'))[5:7]+str(t+np.timedelta64(1,'D'))[8:10]+'T000000Z'
    spelldur_day[['warm_flags', 'cold_flags', 'warm_spell_dur_5days', 'cold_spell_dur_5days', 'warm_spell_dur_10days', 'cold_spell_dur_10days']].to_netcdf(dir +'MarineHeatawaves/cmems_ostia_clim/flags/flag_dur_5_10_daily2_{0}.nc'.format(str(t)[0:4]+str(t)[5:7]+str(t)[8:10]))


20210101
20210102
20210103
20210104
20210105
20210106
20210107
20210108
20210109
20210110
20210111
20210112
20210113
20210114
20210115
20210116
20210117
20210118
20210119
20210120
20210121
20210122
20210123
20210124
20210125
20210126
20210127
20210128
20210129
20210130
20210131
20210201
20210202
20210203
20210204
20210205
20210206
20210207
20210208
20210209
20210210
20210211
20210212
20210213
20210214
20210215
20210216
20210217
20210218
20210219
20210220
20210221
20210222
20210223
20210224
20210225
20210226
20210227
20210228
20210301
20210302
20210303
20210304
20210305
20210306
20210307
20210308
20210309
20210310
20210311
20210312
20210313
20210314
20210315
20210316
20210317
20210318
20210319
20210320
20210321
20210322
20210323
